In [85]:
from utils import CIFAR10, CIFAR100

In [ ]:
print("Reading dataset...")
train_data = CIFAR10(train=True)
test_data  = CIFAR10(train=False)

Xtrain, Ytrain = train_data.train_data, train_data.train_labels
Xtest, Ytest = test_data.test_data, test_data.test_labels

In [83]:
"""
A pure TensorFlow implementation of a neural network. This can be
used as a drop-in replacement for a Keras model.
"""

import numpy as np
import tensorflow as tf
from cleverhans.model import Model


class MLP(Model):
    """
    An example of a bare bones multilayer perceptron (MLP) class.
    """

    def __init__(self, layers, input_shape, pretrain_dict=None):
        super(MLP, self).__init__()
        
        self.layer_names = []
        self.layers = layers
        self.input_shape = input_shape
        if isinstance(layers[-1], Softmax):
            layers[-1].name = 'probs'
            layers[-2].name = 'logits'
        else:
            layers[-1].name = 'logits'
            
        for i, layer in enumerate(self.layers):
            if hasattr(layer, 'name'):
                name = layer.name
                print(name)
                if (pretrain_dict is not None) and (name in pretrain_dict.keys()):
                    layer.set_input_shape(input_shape, pretrain = pretrain_dict[name])
                else:
                    layer.set_input_shape(input_shape)
            else:
                name = layer.__class__.__name__ + str(i)
                layer.name = name
                layer.set_input_shape(input_shape)
                
            self.layer_names.append(name)
            input_shape = layer.get_output_shape()

    def fprop(self, x, set_ref=False):
        states = []
        for layer in self.layers:
            if set_ref:
                layer.ref = x
            x = layer.fprop(x)
            assert x is not None
            states.append(x)
        states = dict(zip(self.get_layer_names(), states))
        return states


class Layer(object):

    def get_output_shape(self):
        return self.output_shape


class Linear(Layer):

    def __init__(self, num_hid=None, **kwargs):
        self.__dict__.update(kwargs)
        self.num_hid = num_hid

    def set_input_shape(self, input_shape, pretrain=None):
        batch_size, dim = input_shape
        if pretrain is None:
            if self.num_hid is None: 
                print("Without pre-trained models, please specify num_hid in Linear Layer.")
            init = tf.random_normal([dim, self.num_hid], dtype=tf.float32)
            init = init / tf.sqrt(1e-7 + tf.reduce_sum(tf.square(init), axis=0, keep_dims=True))
            self.W = tf.Variable(init)
            self.b = tf.Variable(np.zeros((self.num_hid,)).astype('float32'))
            
        else:
            self.W = tf.Variable(initial_value = pretrain[0], dtype = tf.float32)
            self.b = tf.Variable(initial_value = pretrain[1], dtype = tf.float32)
            self.num_hid = len(pretrain[1]) 
        
        self.input_shape = [batch_size, dim]
        self.output_shape = [batch_size, self.num_hid]

    def fprop(self, x):
        return tf.matmul(x, self.W) + self.b


class Conv2D(Layer):

    def __init__(self, output_channels=None, kernel_shape=None, strides=[1,1], padding='SAME', **kwargs):
        self.__dict__.update(kwargs)
        self.__dict__.update(locals())
        del self.self

    def set_input_shape(self, input_shape, pretrain=None):
        batch_size, rows, cols, input_channels = input_shape
        if pretrain is None:
            #assert ((self.kernel_shape is None) or (self.output_channels is None)),\
            #        "Without a pre-trained model, please specify kernel_shape and output_channels in Conv2D layer"
        
            kernel_shape = tuple(self.kernel_shape) + (input_channels, self.output_channels)
            assert len(kernel_shape) == 4
            assert all(isinstance(e, int) for e in kernel_shape), kernel_shape

            init = tf.random_normal(kernel_shape, dtype=tf.float32)
            init = init / tf.sqrt(1e-7 + tf.reduce_sum(tf.square(init), axis=(0, 1, 2)))
            self.kernels = tf.Variable(init)
            self.b = tf.Variable(np.zeros((self.output_channels,)).astype('float32'))
        else:
            self.kernels = tf.Variable(initial_value = pretrain[0], dtype = tf.float32)
            self.b = tf.Variable(initial_value = pretrain[1], dtype = tf.float32)
            self.kernel_shape = tuple(pretrain[0].shape[:2])
            self.output_channels = tuple(pretrain[0].shape[2:3])
        
        input_shape = list(input_shape)
        input_shape[0] = 1
        dummy_batch = tf.zeros(input_shape)
        dummy_output = self.fprop(dummy_batch)
        output_shape = [int(e) for e in dummy_output.get_shape()]
        output_shape[0] = batch_size
        self.output_shape = tuple(output_shape)

    def fprop(self, x):
        return tf.nn.conv2d(x, self.kernels, (1,) + tuple(self.strides) + (1,),
                            self.padding) + self.b


class ReLU(Layer):

    def __init__(self):
        pass

    def set_input_shape(self, shape):
        self.input_shape = shape
        self.output_shape = shape

    def fprop(self, x):
        return tf.nn.relu(x)


class Softmax(Layer):

    def __init__(self):
        pass

    def set_input_shape(self, shape):
        self.input_shape = shape
        self.output_shape = shape

    def fprop(self, x):
        return tf.nn.softmax(x)
    
class MaxPool(Layer):
    
    def __init__(self):
        pass
    
    def set_input_shape(self, shape):
        batch_size, rows, cols, input_channels = shape
        input_shape = list(shape)
        input_shape[0] = 1
        dummy_batch = tf.zeros(input_shape)
        dummy_output = self.fprop(dummy_batch)
        output_shape = [int(e) for e in dummy_output.get_shape()]
        output_shape[0] = batch_size
        
        self.input_shape = shape
        self.output_shape = tuple(output_shape)
        
    def fprop(self, x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


class Flatten(Layer):

    def __init__(self):
        pass

    def set_input_shape(self, shape):
        self.input_shape = shape
        output_width = 1
        for factor in shape[1:]:
            output_width *= factor
        self.output_width = output_width
        self.output_shape = [shape[0], output_width]

    def fprop(self, x):
        return tf.reshape(x, [-1, self.output_width])


def make_basic_cnn(nb_filters=64, nb_classes=10,
                   input_shape=(None, 28, 28, 1)):
    layers = [Conv2D(output_channels=nb_filters, kernel_shape=(8, 8), strides=(2, 2), padding="SAME"),
              ReLU(),
              Conv2D(output_channels=nb_filters * 2, kernel_shape=(6, 6), strides=(2, 2), padding="SAME"),
              ReLU(),
              Conv2D(output_channels=nb_filters * 2, kernel_shape=(5, 5), strides=(1, 1), padding="SAME"),
              ReLU(),
              Flatten(),
              Linear(num_hid=nb_classes),
              Softmax()]

    model = MLP(layers, input_shape)
    return model

def make_pretrain_vgg16(input_shape=(None, 28, 28, 1), pretrain_dict=None):
    layers = [Conv2D(name='conv1_1'),
                ReLU(),
                Conv2D(name='conv1_2'),
                ReLU(),
                Conv2D(name='conv2_1'),
                ReLU(),
                Conv2D(name='conv2_2'),
                ReLU(),
                Conv2D(name='conv3_1'),
                ReLU(),
                Conv2D(name='conv3_2'),
                ReLU(),
                Conv2D(name='conv3_3'),
                ReLU(),
                Conv2D(name='conv4_1'),
                ReLU(),
                Conv2D(name='conv4_2'),
                ReLU(),
                Conv2D(name='conv4_3'),
                ReLU(),
                Conv2D(name='conv5_1'),
                ReLU(),
                Conv2D(name='conv5_2'),
                ReLU(),
                Conv2D(name='conv5_3'),
                ReLU(),
                Flatten(),
                Linear(name='fc6'),
                ReLU(),
                Linear(name='fc7'),
                ReLU(),
                Linear(num_hid=10),
                Softmax()]
    model = MLP(layers, input_shape,pretrain_dict)
    return model
#def make_reduced_vgg16():
    
    

In [84]:
model = make_pretrain_vgg16(input_shape=(None, 28, 28, 3), pretrain_dict=a)

conv1_1
conv1_2
conv2_1
conv2_2
conv3_1
conv3_2
conv3_3
conv4_1
conv4_2
conv4_3
conv5_1
conv5_2
conv5_3
fc6
fc7
logits
probs


In [55]:
model = make_basic_cnn()

In [56]:
model.layers

In [33]:
model.layer_names

['conv1',
 'ReLU1',
 'Conv2D2',
 'ReLU3',
 'Conv2D4',
 'ReLU5',
 'Flatten6',
 'logits',
 'probs']

In [ ]:
"""
This tutorial shows how to generate adversarial examples using FGSM
and train a model using adversarial training with TensorFlow.
It is very similar to mnist_tutorial_keras_tf.py, which does the same
thing but with a dependence on keras.
The original paper can be found at:
https://arxiv.org/abs/1412.6572
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy as np
import tensorflow as tf
from tensorflow.python.platform import flags
import logging

from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import model_train, model_eval
from cleverhans.attacks import FastGradientMethod
from cleverhans_tutorials.tutorial_models import make_basic_cnn
from cleverhans.utils import AccuracyReport, set_log_level

import os

FLAGS = flags.FLAGS


def mnist_tutorial(train_start=0, train_end=60000, test_start=0,
                   test_end=10000, nb_epochs=6, batch_size=128,
                   learning_rate=0.001,
                   clean_train=True,
                   testing=False,
                   backprop_through_attack=False,
                   nb_filters=64, num_threads=None):
    """
    MNIST cleverhans tutorial
    :param train_start: index of first training set example
    :param train_end: index of last training set example
    :param test_start: index of first test set example
    :param test_end: index of last test set example
    :param nb_epochs: number of epochs to train model
    :param batch_size: size of training batches
    :param learning_rate: learning rate for training
    :param clean_train: perform normal training on clean examples only
                        before performing adversarial training.
    :param testing: if true, complete an AccuracyReport for unit tests
                    to verify that performance is adequate
    :param backprop_through_attack: If True, backprop through adversarial
                                    example construction process during
                                    adversarial training.
    :param clean_train: if true, train on clean examples
    :return: an AccuracyReport object
    """

    # Object used to keep track of (and return) key accuracies
    report = AccuracyReport()

    # Set TF random seed to improve reproducibility
    tf.set_random_seed(1234)

    # Set logging level to see debug information
    set_log_level(logging.DEBUG)

    # Create TF session
    if num_threads:
        config_args = dict(intra_op_parallelism_threads=1)
    else:
        config_args = {}
    sess = tf.Session(config=tf.ConfigProto(**config_args))

    # Get MNIST test data
    X_train, Y_train, X_test, Y_test = data_mnist(train_start=train_start,
                                                  train_end=train_end,
                                                  test_start=test_start,
                                                  test_end=test_end)

    # Use label smoothing
    assert Y_train.shape[1] == 10
    label_smooth = .1
    Y_train = Y_train.clip(label_smooth / 9., 1. - label_smooth)

    # Define input TF placeholder
    x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
    y = tf.placeholder(tf.float32, shape=(None, 10))

    model_path = "models/mnist"
    # Train an MNIST model
    train_params = {
        'nb_epochs': nb_epochs,
        'batch_size': batch_size,
        'learning_rate': learning_rate
    }
    fgsm_params = {'eps': 0.3,
                   'clip_min': 0.,
                   'clip_max': 1.}
    rng = np.random.RandomState([2017, 8, 30])

    if clean_train:
        model = make_basic_cnn(nb_filters=nb_filters)
        preds = model.get_probs(x)

        def evaluate():
            # Evaluate the accuracy of the MNIST model on legitimate test
            # examples
            eval_params = {'batch_size': batch_size}
            acc = model_eval(
                sess, x, y, preds, X_test, Y_test, args=eval_params)
            report.clean_train_clean_eval = acc
            assert X_test.shape[0] == test_end - test_start, X_test.shape
            print('Test accuracy on legitimate examples: %0.4f' % acc)
        model_train(sess, x, y, preds, X_train, Y_train, evaluate=evaluate,
                    args=train_params, rng=rng)

        # Calculate training error
        if testing:
            eval_params = {'batch_size': batch_size}
            acc = model_eval(
                sess, x, y, preds, X_train, Y_train, args=eval_params)
            report.train_clean_train_clean_eval = acc

        # Initialize the Fast Gradient Sign Method (FGSM) attack object and
        # graph
        fgsm = FastGradientMethod(model, sess=sess)
        adv_x = fgsm.generate(x, **fgsm_params)
        preds_adv = model.get_probs(adv_x)

        # Evaluate the accuracy of the MNIST model on adversarial examples
        eval_par = {'batch_size': batch_size}
        acc = model_eval(sess, x, y, preds_adv, X_test, Y_test, args=eval_par)
        print('Test accuracy on adversarial examples: %0.4f\n' % acc)
        report.clean_train_adv_eval = acc

        # Calculate training error
        if testing:
            eval_par = {'batch_size': batch_size}
            acc = model_eval(sess, x, y, preds_adv, X_train,
                             Y_train, args=eval_par)
            report.train_clean_train_adv_eval = acc

        print("Repeating the process, using adversarial training")
    # Redefine TF model graph
    model_2 = make_basic_cnn(nb_filters=nb_filters)
    preds_2 = model_2(x)
    fgsm2 = FastGradientMethod(model_2, sess=sess)
    adv_x_2 = fgsm2.generate(x, **fgsm_params)
    if not backprop_through_attack:
        # For the fgsm attack used in this tutorial, the attack has zero
        # gradient so enabling this flag does not change the gradient.
        # For some other attacks, enabling this flag increases the cost of
        # training, but gives the defender the ability to anticipate how
        # the atacker will change their strategy in response to updates to
        # the defender's parameters.
        adv_x_2 = tf.stop_gradient(adv_x_2)
    preds_2_adv = model_2(adv_x_2)

    def evaluate_2():
        # Accuracy of adversarially trained model on legitimate test inputs
        eval_params = {'batch_size': batch_size}
        accuracy = model_eval(sess, x, y, preds_2, X_test, Y_test,
                              args=eval_params)
        print('Test accuracy on legitimate examples: %0.4f' % accuracy)
        report.adv_train_clean_eval = accuracy

        # Accuracy of the adversarially trained model on adversarial examples
        accuracy = model_eval(sess, x, y, preds_2_adv, X_test,
                              Y_test, args=eval_params)
        print('Test accuracy on adversarial examples: %0.4f' % accuracy)
        report.adv_train_adv_eval = accuracy

    # Perform and evaluate adversarial training
    model_train(sess, x, y, preds_2, X_train, Y_train,
                predictions_adv=preds_2_adv, evaluate=evaluate_2,
                args=train_params, rng=rng)

    # Calculate training errors
    if testing:
        eval_params = {'batch_size': batch_size}
        accuracy = model_eval(sess, x, y, preds_2, X_train, Y_train,
                              args=eval_params)
        report.train_adv_train_clean_eval = accuracy
        accuracy = model_eval(sess, x, y, preds_2_adv, X_train,
                              Y_train, args=eval_params)
        report.train_adv_train_adv_eval = accuracy

    return report


def main(argv=None):
    mnist_tutorial(nb_epochs=FLAGS.nb_epochs, batch_size=FLAGS.batch_size,
                   learning_rate=FLAGS.learning_rate,
                   clean_train=FLAGS.clean_train,
                   backprop_through_attack=FLAGS.backprop_through_attack,
                   nb_filters=FLAGS.nb_filters)


if __name__ == '__main__':
    flags.DEFINE_integer('nb_filters', 64, 'Model size multiplier')
    flags.DEFINE_integer('nb_epochs', 6, 'Number of epochs to train model')
    flags.DEFINE_integer('batch_size', 128, 'Size of training batches')
    flags.DEFINE_float('learning_rate', 0.001, 'Learning rate for training')
    flags.DEFINE_bool('clean_train', True, 'Train on clean examples')
    flags.DEFINE_bool('backprop_through_attack', False,
                      ('If True, backprop through adversarial example '
                       'construction process during adversarial training'))

    tf.app.run()
© 2018 GitHub, Inc.
Terms
Privacy
Security
Status
Help
Contact GitHub
API
Training
Shop
Blog
About
Press h to open a hovercard with more details.

In [ ]:
def idp_conv_bn_layer(self, bottom, name, dp=1.0):
        with tf.name_scope(name+str(int(dp*100))):
            with tf.variable_scope("VGG16",reuse=True):
                conv_filter = tf.get_variable(name=name+"_W")
                conv_biases = tf.get_variable(name=name+"_b")
                conv_gamma  = tf.get_variable(name=name+"_gamma")
                moving_mean = tf.get_variable(name=name+'_bn_mean')
                moving_variance = tf.get_variable(name=name+'_bn_variance')
                beta = tf.get_variable(name=name+'_beta')
            H,W,C,O = conv_filter.get_shape().as_list()
            
            # create a mask determined by the dot product percentage
            n1 = int(O * dp)
            n0 = O - n1
            mask = tf.constant(value=np.append(np.ones(n1, dtype='float32'), np.zeros(n0, dtype='float32')), dtype=tf.float32)
            conv_gamma = tf.multiply(conv_gamma, mask)
            beta = tf.multiply(beta, mask)
            
            conv = tf.nn.conv2d(bottom, conv_filter, [1, 1, 1, 1], padding='SAME')
            conv = tf.nn.bias_add(conv, conv_biases)

            from tensorflow.python.training.moving_averages import assign_moving_average
            def mean_var_with_update():
                mean, variance = tf.nn.moments(conv, [0,1,2], name='moments')
                with tf.control_dependencies([assign_moving_average(moving_mean, mean, 0.9),
                                              assign_moving_average(moving_variance, variance, 0.9)]):
                    return tf.identity(mean), tf.identity(variance)

            mean, variance = tf.cond(self.is_train, mean_var_with_update, lambda:(moving_mean, moving_variance))

            conv = tf.nn.batch_normalization(conv, mean, variance, beta, conv_gamma, 1e-05)
            relu = tf.nn.relu(conv)
            
            return relu